In [1]:
%pip install datasets langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


# Dataset Loading

- [Naver 경제, IT 뉴스기사 요약 데이터셋](https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko)

In [2]:
from datasets import load_dataset
import pandas as pd

# Hugging Face 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

c:\Users\Playdata\AppData\Local\miniconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--daekeun-ml--naver-news-summarization-ko. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [3]:
trainset = dataset['train']
trainset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 22194
})

In [4]:
df_train = trainset.to_pandas()
df_train.shape

(22194, 7)

In [5]:
df = df_train.query("category=='economy'").reset_index(drop=True)
df.shape

(17088, 7)

# 데이터셋 만들기
- 뉴스기사 제목과 뉴스기사를 이용해 그 기사에 영향을 받는 주가종목을 추론하는 모델을 만든다.
- 데이터 구성
  - **입력**: 뉴스 기사 제목, 뉴스 기사
  - **출력**: 뉴스기사가 주식에 영향을 주는지 여부, 부정적인 영향을 받는 회사와 이유, 긍정적인 영향을 받는 회사와 이유, 뉴스기사 요약
- Label을 LLM을 이용해 생성한다.
  - LLM을 이용해 데이터셋을 만든 이후 그 결과를 눈으로 검토해야 한다.

## Dataset 생성 Chain 구성

In [6]:
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
template = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - 뉴스 요약 (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - 뉴스 요약 (3줄 이내)

# 입력 데이터(뉴스기사)

{input}


# 출력 지시사항 (Output Indicator)

- {format_instructions}

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{{'is_stock_related': true,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {{"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"}},
        {{"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}}'''

In [10]:
class SummarySchema(BaseModel):
    is_stock_related: bool = Field(description="한국 주식과 관련있는 뉴스인지 여부")
    positive_stocks: list[str] = Field(description="뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.")
    positive_reason: list[dict[str,str]] = Field(description='뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {"회사이름":"긍정적인 이유"}')
    negative_stocks: list[str] = Field(description="뉴스기사에 부정적인 영향을 받는 회사들의 이름들.")
    negative_reason: list[dict[str,str]] = Field(description='뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {"회사이름":"부정적인 이유"}')
    summary: str = Field(description="뉴스기사 요약")

parser = JsonOutputParser(pydantic_object=SummarySchema)

prompt_template = ChatPromptTemplate(
    messages=[
        ("user", template), 
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_name = "gpt-4.1"
chain = prompt_template | ChatOpenAI(model=model_name) | parser

### 개별 데이터로 Chain 테스트

In [15]:
from pprint import pprint

news_idx = 98
sample_news = df['title'].loc[news_idx]+"\n"+df['document'].loc[news_idx]
print(sample_news)

카카오페이·증권 결제 스탬프 받고 랜덤 주식도 받으세요
카카오페이와 카카오페이증권이 이달 진행하는 춘식이 심부름 스탬프 프로모션 포스터 ⓒ카카오페이 데일리안 김효숙 기자 카카오페이와 카카오페이증권은 춘식이 심부름 스탬프 프로모션을 새 단장하고 참가자들에게 랜덤 주식과 카카오페이포인트를 선물한다고 1일 밝혔다. 양사는 사용자들이 카카오페이에서 다채로운 금융 서비스를 경험할 수 있도록 결제 프로모션과 금융상품의 연계를 강화해 나간다는 방침이다. 카카오페이와 카카오페이증권은 이날부터 31일까지 한 달간 대표 업종 5곳 등에서 결제 서비스를 이용하고 스탬프를 모으는 사용자들에게 2천 원 상당의 랜덤 주식과 8000 카카오페이포인트를 제공한다. 먼저 스탬프 2개를 모은 사용자들에게는 카카오페이증권에서 2000원 상당의 애플 테슬라 등 미국 증권시장 주요 기업 주식을 랜덤으로 선물한다. 스탬프 3개를 모으면 1000포인트를 5개를 모으면 2000포인트를 더 받게 되며 7개를 모두 모으면 추가로 5000포인트를 받을 수 있다. 모을 수 있는 스탬프의 유형은 총 7가지로 업종 스탬프 5종과 스페셜 스탬프 2종이다. 이번 스탬프 프로모션에는 카카오 카카오 선물하기 외 13개 대표 카카오 가맹점 편의점 GS25 CU 세븐일레븐 이마트24 미니스톱 스토리웨이 패션 무신사 지그재그 크림 SSF 브랜디 나이키 등 공식 온라인 스토어 결제시 배달 배달의민족 요기요 숙박・여행 야놀자 여기어때 등 총 5개 업종이 포함됐다. 페이앱을 통해 결제하거나 배스킨라빈스에서 결제하면 각각의 스페셜 스탬프를 받을 수 있다. 스탬프는 해당 조건에 맞는 최초 결제 시 지급되며 카카오페이 사용자가 편의점에서 페이앱으로 결제할 경우 편의점 업종 스탬프와 페이앱 결제 스페셜 스탬프를 같이 받는 식이다. 카카오페이는 이달 프로모션 기간 동안 배스킨라빈스 매장에서 버라이어티팩을 비롯한 아이스크림 기프트팩 3종을 카카오페이머니로 결제할 경우 5000원의 혜택도 제공한다. 사용자들은 스페셜 스탬프를 모으는 동시에 추

In [16]:
# 4.1 결과
response = chain.invoke(input={"input":sample_news})

pprint(response)

{'is_stock_related': True,
 'negative_reason': [],
 'negative_stocks': [],
 'positive_reason': [{'카카오페이': '춘식이 심부름 스탬프 프로모션을 통해 사용자 참여와 결제 경험을 확대하고, '
                               '금융서비스와 결제의 연계를 강화한다는 소식이 긍정적으로 작용'},
                     {'카카오페이증권': '카카오페이와 함께 랜덤 주식 증정 등 프로모션으로 금융상품 이용자 유입 효과 '
                                 '기대'}],
 'positive_stocks': ['카카오페이', '카카오페이증권'],
 'summary': '카카오페이와 카카오페이증권이 결제 및 금융상품 연계 스탬프 프로모션을 실시해, 참가자들에게 랜덤 주식과 포인트를 '
            '제공한다. 여러 업종과 연계된 이번 이벤트로 이용자 경험 확대를 노린다. 카카오페이는 다양한 금융 서비스 제공을 '
            '목표로 한다.'}


## 답(label) 만들기

1. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 입력데이터를 만든다.
2. 2의  입력데이터를 LLM에 요청해서 답변을 받은 뒤 DataFrame에 추가한다.

In [17]:
# 1. K개 샘플링

sample_nums = 100  
sample_df = df.sample(sample_nums).reset_index(drop=True)
sample_df.shape

(100, 7)

In [18]:
# 2. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 프롬프트를 생성한다.

articles = sample_df['title']+"\n"+sample_df['document']
articles.shape

(100,)

In [19]:
# 3. LLM에 label 생성 요청

label_list = articles.apply(lambda x : chain.invoke({'input':x}))

In [20]:
# Label 타입 변환:  Dictionary 를 str로 변환.

label_list_str = label_list.apply(lambda x : str(x))
type(label_list_str[0]), label_list_str[0]

(str,
 "{'is_stock_related': False, 'positive_stocks': [], 'positive_reasons': [], 'negative_stocks': [], 'negative_reasons': [], 'summary': '감사원이 세종시 특별공급 과정의 부정행위 45건을 적발해 주택 환수 및 고발조치에 나선다. 사업자에 대한 추가 조사와 조치도 예고됐다. 정부는 향후 주택 공급시장 교란행위 점검을 강화할 계획이다.'}")

In [21]:
# Label을 데이터 프레임에 추가.
sample_df['label'] = label_list_str
sample_df.head()

,date,category,press,title,document,link,summary,label
0,2022-07-06 16:46:01,economy,경향신문,감사원 세종시 특공 부정행위 45건 적발…국토부 “주택환수 및 고발조치”,원희룡 국토부 장관이 지난 5일 서울 종로구 정부서울청사 브리핑실에서 산하 공공기관...,https://n.news.naver.com/mnews/article/032/000...,지난토교통부는 정부청사를 세종시로 이전하면서 직원들을 대상으로 실시했던 아파트 특별...,"{'is_stock_related': False, 'positive_stocks':..."
1,2022-07-05 10:34:10,economy,아시아경제,포토 실물 공개되는 쌍용 토레스,쌍용자동차가 5일 인천 영종도 네스트호텔에서 신차발표회를 열고 새 중형 스포츠유틸리...,https://n.news.naver.com/mnews/article/277/000...,쌍용자동차가 5일 인천 영종도 네스트호텔에서 신차발표회를 열고 최대토크 28.5kg...,"{'is_stock_related': True, 'positive_stocks': ..."
2,2022-07-06 16:59:30,economy,MBN,미래에너지 기술 한 눈에 본다…SWEET 2022 개막,SWEET 2022 오늘 6일 김대중컨벤션센터에서 개막…사흘간의 일정 돌입 한전 등...,https://n.news.naver.com/mnews/article/057/000...,국제 6일 광주 김대중컨벤션센터에서 개막한 국제 신재생에너지 전시회 SWEET 20...,"{'is_stock_related': True, 'positive_stocks': ..."
3,2022-07-06 16:30:03,economy,서울경제,금융위 다우키움 금융복합기업집단 추가 지정…총 7곳,삼성·한화·미래에셋·교보·현대차·DB·다우키움 등 지난해 7월에 이은 두 번째 지정...,https://n.news.naver.com/mnews/article/011/000...,금융위원회는 6일 정례회의에서 삼성·한화·미래에셋·교보·현대차·DB·다우키움 자산 ...,"{'is_stock_related': True, 'positive_stocks': ..."
4,2022-07-03 14:34:14,economy,뉴스1,추경호 수출 중소·중견기업 무역금융 계획대비 40조 이상 확대,서울 뉴스1 송원영 기자 추경호 경제부총리 겸 기획재정부 장관이 3일 오후 서울 종...,https://n.news.naver.com/mnews/article/421/000...,3일 추경호 경제부총리 겸 기획재정부 추경호 경제부총리 겸 기획재정부 장관이 3일 ...,"{'is_stock_related': False, 'positive_stocks':..."


In [22]:
############ pickle로 저장
import os
os.makedirs("dataset", exist_ok=True)

output_file = "dataset/sample_df.pkl"

sample_df.to_pickle(
    output_file
)

# import pandas as pd
# sample_df = pd.read_pickle(output_file)

# 허깅페이스에 업로드

In [23]:
from huggingface_hub import login
from datasets import load_dataset

from dotenv import load_dotenv

load_dotenv()

True

In [24]:
import os
login(os.getenv("HUGGINGFACE_API_KEY"))

In [25]:
from datasets import Dataset

# Dataset으로 변환.
dataset = Dataset.from_pandas(sample_df)
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
    num_rows: 100
})

In [26]:
# train/valid/test set으로 분리
dataset_dict = dataset.train_test_split(test_size=0.1)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [28]:
# 데이터셋을 Huggingface hub 에 업로드.
dataset_id = "naver_economy_news_stock_instruct_dataset"
dataset_dict.push_to_hub(dataset_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/192k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/27.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/playdatakoo/naver_economy_news_stock_instruct_dataset/commit/86a8f68600ae496c61b94c2e62d79d343d8dcc16', commit_message='Upload dataset', commit_description='', oid='86a8f68600ae496c61b94c2e62d79d343d8dcc16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/playdatakoo/naver_economy_news_stock_instruct_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='playdatakoo/naver_economy_news_stock_instruct_dataset'), pr_revision=None, pr_num=None)

In [30]:
# dataset load

load_data = load_dataset("playdatakoo/naver_economy_news_stock_instruct_dataset")
load_data

README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

c:\Users\Playdata\AppData\Local\miniconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--playdatakoo--naver_economy_news_stock_instruct_dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00001.parquet:   0%|          | 0.00/192k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/27.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [31]:
d_id = "playdatakoo/naver_economy_news_stock_instruct_dataset"
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
    num_rows: 100
})